In [6]:
#import the necessary libraries
from os import listdir
from os.path import isfile, join
from skimage.transform import resize
import numpy as np
import cv2 as cv
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage import exposure
import csv
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from skimage import feature
from sklearn.metrics import confusion_matrix
import csv
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from skimage.feature import hog
%matplotlib inline

In [7]:
#path where images are located
mypath='UFPR-Periocular/periocularCropped/'
#Reading files of images from folder path
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)

In [8]:
#Checking age range
with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender
    age_groups = pd.cut(df['age'], bins=[0,35,55,99])
    df.groupby(age_groups)
    print(pd.crosstab(age_groups, df['gender']))

gender    female   male
age                    
(0, 35]    10440  13590
(35, 55]    3930   3600
(55, 99]    1230    870


In [15]:
mypath='UFPR-Periocular/periocularCropped/'
columns = defaultdict(list)
age_label_18_35 = []
lbp_features_18_35 = []
age_label_36_55 = []
lbp_features_36_55 = []
age_label_56_ = []
lbp_features_56_ = []


with open('UFPR-Periocular/imagesInfo_csv.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    df = pd.read_csv(csv_file)
    image_file_name,age,gender = df.file_name,df.age,df.gender

    print(len(image_file_name))
    for i in range(len(image_file_name)):
        image = cv.imread( join(mypath,image_file_name[i]) )
        img = cv.cvtColor(image ,cv.COLOR_RGB2BGR)
        #Resizing the images
        resized_img = resize(img, (128,64)) 
        #creating hog features for the data
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                        cells_per_block=(2, 2), visualize=True, multichannel=True)

        #Set age range label after gender bias
        #appending the hog features for image onto the hog data lists 
        #hog_features.append(fd)
        
        
        if age[i] >= 10 and age[i]<=35:
            lbp_features_18_35.append(fd)
            if gender[i]== 'male':
                age_label_18_35.append("male")
            else:
                age_label_18_35.append("female")
        elif age[i]>36 and age[i] <=55:    
            lbp_features_36_55.append(fd)
            if gender[i]== 'male':
                age_label_36_55.append("male")
            else:
                age_label_36_55.append("female")
        elif age[i]>55:    
            lbp_features_56_.append(fd)
            if gender[i]== 'male':
                age_label_56_.append("male")
            else:
                age_label_56_.append("female") 
                
            

33660


# Age Range 18-35

In [16]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_18_35)
X =  np.array(lbp_features_18_35)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 18-35")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 18-35
Overall Accuracy using Linear SVM: 0.9173533083645443
              precision    recall  f1-score   support

      female       0.92      0.89      0.90      5235
        male       0.92      0.94      0.93      6780

    accuracy                           0.92     12015
   macro avg       0.92      0.91      0.92     12015
weighted avg       0.92      0.92      0.92     12015

Accuracy for each class
[0.89111748 0.93761062]




Non-Linear RBF Accuracy: 0.5642946317103621
              precision    recall  f1-score   support

      female       0.00      0.00      0.00      5235
        male       0.56      1.00      0.72      6780

    accuracy                           0.56     12015
   macro avg       0.28      0.50      0.36     12015
weighted avg       0.32      0.56      0.41     12015

              precision    recall  f1-score   support

      female       0.00      0.00      0.00      5235
        male       0.56      1.00      0.72      6780

    accuracy    

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Accuracy for each class
[0. 1.]




Non-Linear Poly Accuracy: 0.9449854348730753
              precision    recall  f1-score   support

      female       0.94      0.93      0.94      5235
        male       0.95      0.95      0.95      6780

    accuracy                           0.94     12015
   macro avg       0.94      0.94      0.94     12015
weighted avg       0.94      0.94      0.94     12015

              precision    recall  f1-score   support

      female       0.94      0.93      0.94      5235
        male       0.95      0.95      0.95      6780

    accuracy                           0.94     12015
   macro avg       0.94      0.94      0.94     12015
weighted avg       0.94      0.94      0.94     12015

Accuracy for each class
[0.93352436 0.95383481]




Iteration 1, loss = 0.68868719
Iteration 2, loss = 0.67164592
Iteration 3, loss = 0.65852226
Iteration 4, loss = 0.64321215
Iteration 5, loss = 0.62521298
Iteration 6, loss = 0.60357154
Iteration 7, loss = 0.57933

Iteration 229, loss = 0.00355080
Iteration 230, loss = 0.00345200
Iteration 231, loss = 0.00340986
Iteration 232, loss = 0.00339170
Iteration 233, loss = 0.00332155
Iteration 234, loss = 0.00327362
Iteration 235, loss = 0.00323883
Iteration 236, loss = 0.00318200
Iteration 237, loss = 0.00324222
Iteration 238, loss = 0.00311555
Iteration 239, loss = 0.00307836
Iteration 240, loss = 0.00304246
Iteration 241, loss = 0.00303635
Iteration 242, loss = 0.00298693
Iteration 243, loss = 0.00294250
Iteration 244, loss = 0.00290283
Iteration 245, loss = 0.00288353
Iteration 246, loss = 0.00283292
Iteration 247, loss = 0.00283685
Iteration 248, loss = 0.00278997
Iteration 249, loss = 0.00274670
Iteration 250, loss = 0.00270729
Iteration 251, loss = 0.00268720
Iteration 252, loss = 0.00266095
Iteration 253, loss = 0.00262045
Iteration 254, loss = 0.00260697
Iteration 255, loss = 0.00257121
Iteration 256, loss = 0.00255245
Iteration 257, loss = 0.00253647
Iteration 258, loss = 0.00250591
Iteration 

Iteration 478, loss = 0.00071548
Iteration 479, loss = 0.00071199
Iteration 480, loss = 0.00071309
Iteration 481, loss = 0.00071011
Iteration 482, loss = 0.00070684
Iteration 483, loss = 0.00070278
Iteration 484, loss = 0.00070216
Iteration 485, loss = 0.00069955
Iteration 486, loss = 0.00069901
Iteration 487, loss = 0.00069638
Iteration 488, loss = 0.00069389
Iteration 489, loss = 0.00069015
Iteration 490, loss = 0.00068825
Iteration 491, loss = 0.00068807
Iteration 492, loss = 0.00068354
Iteration 493, loss = 0.00068219
Iteration 494, loss = 0.00067929
Iteration 495, loss = 0.00067772
Iteration 496, loss = 0.00067671
Iteration 497, loss = 0.00067634
Iteration 498, loss = 0.00067020
Iteration 499, loss = 0.00067045
Iteration 500, loss = 0.00066786


/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Accuracy: 0.9165210153974199
              precision    recall  f1-score   support

      female       0.91      0.90      0.90      5235
        male       0.92      0.93      0.93      6780

    accuracy                           0.92     12015
   macro avg       0.92      0.91      0.92     12015
weighted avg       0.92      0.92      0.92     12015

Accuracy for each class
[0.90181471 0.92787611]






# Age Range 36-55

In [17]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_36_55)
X =  np.array(lbp_features_36_55)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range 36-55")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range 36-55
Overall Accuracy using Linear SVM: 0.9141666666666667
              precision    recall  f1-score   support

      female       0.92      0.91      0.92      1890
        male       0.90      0.92      0.91      1710

    accuracy                           0.91      3600
   macro avg       0.91      0.91      0.91      3600
weighted avg       0.91      0.91      0.91      3600

Accuracy for each class
[0.91111111 0.91754386]




Non-Linear RBF Accuracy: 0.525
              precision    recall  f1-score   support

      female       0.53      1.00      0.69      1890
        male       0.00      0.00      0.00      1710

    accuracy                           0.53      3600
   macro avg       0.26      0.50      0.34      3600
weighted avg       0.28      0.53      0.36      3600

              precision    recall  f1-score   support

      female       0.53      1.00      0.69      1890
        male       0.00      0.00      0.00      1710

    accuracy                 

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.945
              precision    recall  f1-score   support

      female       0.95      0.95      0.95      1890
        male       0.94      0.94      0.94      1710

    accuracy                           0.94      3600
   macro avg       0.94      0.94      0.94      3600
weighted avg       0.95      0.94      0.95      3600

              precision    recall  f1-score   support

      female       0.95      0.95      0.95      1890
        male       0.94      0.94      0.94      1710

    accuracy                           0.94      3600
   macro avg       0.94      0.94      0.94      3600
weighted avg       0.95      0.94      0.95      3600

Accuracy for each class
[0.94656085 0.94327485]




Iteration 1, loss = 0.69014701
Iteration 2, loss = 0.68832930
Iteration 3, loss = 0.68628428
Iteration 4, loss = 0.68411035
Iteration 5, loss = 0.68188447
Iteration 6, loss = 0.67953059
Iteration 7, loss = 0.67693750
Iteration 8, loss = 0.67435373
Iteration 9, l

Iteration 231, loss = 0.03237413
Iteration 232, loss = 0.03193065
Iteration 233, loss = 0.03116008
Iteration 234, loss = 0.03054477
Iteration 235, loss = 0.02991555
Iteration 236, loss = 0.02958584
Iteration 237, loss = 0.02898556
Iteration 238, loss = 0.02897471
Iteration 239, loss = 0.02821450
Iteration 240, loss = 0.02792670
Iteration 241, loss = 0.02759562
Iteration 242, loss = 0.02715045
Iteration 243, loss = 0.02652487
Iteration 244, loss = 0.02582576
Iteration 245, loss = 0.02564880
Iteration 246, loss = 0.02550229
Iteration 247, loss = 0.02489931
Iteration 248, loss = 0.02443069
Iteration 249, loss = 0.02426076
Iteration 250, loss = 0.02401907
Iteration 251, loss = 0.02325907
Iteration 252, loss = 0.02297291
Iteration 253, loss = 0.02275403
Iteration 254, loss = 0.02250076
Iteration 255, loss = 0.02218649
Iteration 256, loss = 0.02182597
Iteration 257, loss = 0.02151595
Iteration 258, loss = 0.02146258
Iteration 259, loss = 0.02100168
Iteration 260, loss = 0.02067075
Iteration 

Iteration 480, loss = 0.00289007
Iteration 481, loss = 0.00287244
Iteration 482, loss = 0.00286039
Iteration 483, loss = 0.00285607
Iteration 484, loss = 0.00283562
Iteration 485, loss = 0.00282954
Iteration 486, loss = 0.00280542
Iteration 487, loss = 0.00279982
Iteration 488, loss = 0.00279412
Iteration 489, loss = 0.00277802
Iteration 490, loss = 0.00276764
Iteration 491, loss = 0.00274523
Iteration 492, loss = 0.00273319
Iteration 493, loss = 0.00271458
Iteration 494, loss = 0.00270367
Iteration 495, loss = 0.00267934
Iteration 496, loss = 0.00267945
Iteration 497, loss = 0.00266196
Iteration 498, loss = 0.00265639
Iteration 499, loss = 0.00263913
Iteration 500, loss = 0.00262773
MLP Accuracy: 0.8755555555555555
              precision    recall  f1-score   support

      female       0.88      0.88      0.88      1890
        male       0.87      0.87      0.87      1710

    accuracy                           0.88      3600
   macro avg       0.88      0.88      0.88      3600
we

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Age Range >55

In [18]:
# Fit a SVM classifier to the data where gender=male
y =  np.array(age_label_56_)
X =  np.array(lbp_features_56_)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)
clf = svm.SVC().fit(x_train,y_train)
y_pred = clf.predict(x_test)
#Linear SVM
print("Age Range > 55")
print("Overall Accuracy using Linear SVM: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')
C = 1.0

#Non-Linear RBF
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C)
rbf_svc.fit(x_train,y_train)
y_pred = rbf_svc.predict(x_test)  
print("Non-Linear RBF Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#Non-Linear Ploy
poly_svc = svm.SVC(kernel='poly', degree=3, C=C)
poly_svc.fit(x_train,y_train)
y_pred = poly_svc.predict(x_test)
print("Non-Linear Poly Accuracy: "+str(accuracy_score(y_test, y_pred)))
#print('\n')
print(classification_report(y_test, y_pred))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

#MLP Classifier
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.0001,
                 solver='sgd', verbose=10,  random_state=21,tol=0.000000001)
mlp_clf.fit(x_train, y_train)
y_pred = mlp_clf.predict(x_test)
print("MLP Accuracy: "+str(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print("Accuracy for each class")
print(cm.diagonal()/cm.sum(axis=1))
print('\n')
print('\n')

Age Range > 55
Overall Accuracy using Linear SVM: 0.9419047619047619
              precision    recall  f1-score   support

      female       0.94      0.96      0.95       619
        male       0.94      0.91      0.93       431

    accuracy                           0.94      1050
   macro avg       0.94      0.94      0.94      1050
weighted avg       0.94      0.94      0.94      1050

Accuracy for each class
[0.96122779 0.91415313]




Non-Linear RBF Accuracy: 0.5895238095238096
              precision    recall  f1-score   support

      female       0.59      1.00      0.74       619
        male       0.00      0.00      0.00       431

    accuracy                           0.59      1050
   macro avg       0.29      0.50      0.37      1050
weighted avg       0.35      0.59      0.44      1050

              precision    recall  f1-score   support

      female       0.59      1.00      0.74       619
        male       0.00      0.00      0.00       431

    accuracy     

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metr

Non-Linear Poly Accuracy: 0.960952380952381
              precision    recall  f1-score   support

      female       0.96      0.97      0.97       619
        male       0.96      0.94      0.95       431

    accuracy                           0.96      1050
   macro avg       0.96      0.96      0.96      1050
weighted avg       0.96      0.96      0.96      1050

              precision    recall  f1-score   support

      female       0.96      0.97      0.97       619
        male       0.96      0.94      0.95       431

    accuracy                           0.96      1050
   macro avg       0.96      0.96      0.96      1050
weighted avg       0.96      0.96      0.96      1050

Accuracy for each class
[0.97253635 0.94431555]




Iteration 1, loss = 0.68038685
Iteration 2, loss = 0.67937701
Iteration 3, loss = 0.67803162
Iteration 4, loss = 0.67670269
Iteration 5, loss = 0.67546207
Iteration 6, loss = 0.67437138
Iteration 7, loss = 0.67326952
Iteration 8, loss = 0.67228906
It

Iteration 231, loss = 0.05727278
Iteration 232, loss = 0.05644078
Iteration 233, loss = 0.05552122
Iteration 234, loss = 0.05565405
Iteration 235, loss = 0.05394088
Iteration 236, loss = 0.05402027
Iteration 237, loss = 0.05237075
Iteration 238, loss = 0.05145816
Iteration 239, loss = 0.05097516
Iteration 240, loss = 0.05028987
Iteration 241, loss = 0.04980335
Iteration 242, loss = 0.04895720
Iteration 243, loss = 0.04793111
Iteration 244, loss = 0.04779033
Iteration 245, loss = 0.04668987
Iteration 246, loss = 0.04619297
Iteration 247, loss = 0.04582189
Iteration 248, loss = 0.04542553
Iteration 249, loss = 0.04422293
Iteration 250, loss = 0.04437821
Iteration 251, loss = 0.04302688
Iteration 252, loss = 0.04248145
Iteration 253, loss = 0.04275524
Iteration 254, loss = 0.04150659
Iteration 255, loss = 0.04115561
Iteration 256, loss = 0.04077641
Iteration 257, loss = 0.03989215
Iteration 258, loss = 0.04040938
Iteration 259, loss = 0.03939656
Iteration 260, loss = 0.03840411
Iteration 

Iteration 480, loss = 0.00629499
Iteration 481, loss = 0.00625894
Iteration 482, loss = 0.00622612
Iteration 483, loss = 0.00621055
Iteration 484, loss = 0.00617487
Iteration 485, loss = 0.00615156
Iteration 486, loss = 0.00611274
Iteration 487, loss = 0.00606785
Iteration 488, loss = 0.00604136
Iteration 489, loss = 0.00600967
Iteration 490, loss = 0.00600650
Iteration 491, loss = 0.00596206
Iteration 492, loss = 0.00591626
Iteration 493, loss = 0.00588528
Iteration 494, loss = 0.00586164
Iteration 495, loss = 0.00584013
Iteration 496, loss = 0.00583612
Iteration 497, loss = 0.00577246
Iteration 498, loss = 0.00575894
Iteration 499, loss = 0.00572133
Iteration 500, loss = 0.00571203
MLP Accuracy: 0.919047619047619
              precision    recall  f1-score   support

      female       0.93      0.93      0.93       619
        male       0.90      0.90      0.90       431

    accuracy                           0.92      1050
   macro avg       0.92      0.92      0.92      1050
wei

/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
